In [1]:
import os
os.chdir('../iml-hack')
os.curdir

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler

import data_prep
import utils

from typing import List, Tuple

import xgboost as xgb

In [2]:
si = SimpleImputer(strategy='mean')
knni = KNNImputer(n_neighbors=100)

In [3]:
data_x, data_y = data_prep.load_train(perform=False, group=True)
print(np.isnan(data_x).sum())
data_x = knni.fit_transform(data_x)
print(np.isnan(data_x).sum())
print(data_x.shape, data_y.shape)
data_x = StandardScaler().fit_transform(data_x)

28933
0
(8000, 126) (8000,)


In [4]:
folds = data_prep.prep_crossvalidation(data_x, data_y, batch_size=64, n_splits=5, seed=42, dataloader=False)
train_dataset, val_dataset = folds[0]

In [5]:
train_x, train_y = train_dataset

In [18]:
custom_objective = utils.CustomCMLoss(alpha=1., beta=0.)
xgb_classif = xgb.XGBClassifier(
    objective=custom_objective,
    learning_rate=0.01
    )

fitted = xgb_classif.fit(
    train_x,
    train_y,
)

In [19]:
utils.evaluate_xgb(fitted.predict(val_dataset[0]), val_dataset[1])

1.0875